In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/stores_sales_simple/stores_sales_simple.csv")
df.head()

,unit_id,time,location,size,sales,ads
0,0,2020-01-01,LA,small,62319.650534,ad 2
1,0,2020-01-02,LA,small,61118.764040,ad 2
2,0,2020-01-03,LA,small,52576.959980,ad 2
3,0,2020-01-04,LA,small,34248.010877,ad 2
4,0,2020-01-05,LA,small,24711.595623,ad 2


In [3]:
df[["location", "size"]].drop_duplicates().values

array([['LA', 'small'],
       ['New York', 'large'],
       ['LA', 'large'],
       ['LA', 'medium'],
       ['Boston', 'medium'],
       ['New York', 'small'],
       ['Boston', 'large']], dtype=object)

In [4]:
from algorithms.snn import SNN

In [5]:
model = SNN()

In [6]:
model.fit(df= df, unit_column="unit_id", time_column="time", metrics = ["sales"], actions = ["ads"])

[SNN] iteration 1 of 10000
[SNN] iteration 2 of 10000
[SNN] iteration 3 of 10000
[SNN] iteration 4 of 10000
[SNN] iteration 5 of 10000
[SNN] iteration 6 of 10000
[SNN] iteration 7 of 10000
[SNN] iteration 8 of 10000
[SNN] iteration 9 of 10000
[SNN] iteration 10 of 10000
[SNN] iteration 11 of 10000
[SNN] iteration 12 of 10000
[SNN] iteration 13 of 10000
[SNN] iteration 14 of 10000
[SNN] iteration 15 of 10000
[SNN] iteration 16 of 10000
[SNN] iteration 17 of 10000
[SNN] iteration 18 of 10000
[SNN] iteration 19 of 10000
[SNN] iteration 20 of 10000
[SNN] iteration 21 of 10000
[SNN] iteration 22 of 10000
[SNN] iteration 23 of 10000
[SNN] iteration 24 of 10000
[SNN] iteration 25 of 10000
[SNN] iteration 26 of 10000
[SNN] iteration 27 of 10000
[SNN] iteration 28 of 10000
[SNN] iteration 29 of 10000
[SNN] iteration 30 of 10000
[SNN] iteration 31 of 10000
[SNN] iteration 32 of 10000
[SNN] iteration 33 of 10000
[SNN] iteration 34 of 10000
[SNN] iteration 35 of 10000
[SNN] iteration 36 of 10000
[

In [8]:
model.query([0],["2020-01-10", " 2020-02-19"], 'sales', 'ad 0', ["2020-01-10", " 2020-02-19"])

,2020-01-11,2020-01-12,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18
0,43350.660921,36300.748714,35819.432312,33080.995066,48404.594151,44831.676579,50505.250697,50882.651227,45741.112431,38646.416361,...,41419.247101,51239.03994,47476.411645,44029.544814,39800.191507,36962.170185,31743.557398,35596.343087,41198.359272,48552.395145


In [8]:
model.actions_dict

{'ad 2': 0, 'ad 0': 1, 'ad 1': 2}